# Naive Bayes Baseline with Predetermined Vocabulary

Reference code: CS229 Problem Set 2 spam.py

In [ ]:
import csv
import numpy as np
import pandas as pd
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics

# Training Data

In [ ]:
league_train1_full = pd.read_csv("league_train1.csv", skiprows=[1], usecols = ['league_train1','Unnamed: 2'])
messages1 = league_train1_full['league_train1']
labels1 = league_train1_full['Unnamed: 2']
messages1 = list(messages1)
labels1 = np.asarray(labels1)

In [ ]:
league_test1_full = pd.read_csv("league_test1.csv", skiprows=[1], usecols=['Unnamed: 0', 'Unnamed: 1'])
test_messages1 = league_test1_full['Unnamed: 0']
test_labels1 = league_test1_full['Unnamed: 1']
test_labels1 = np.asarray(test_labels1)
test_messages1 = list(test_messages1)

# Processing Frequencies



*   Create dictionary of frequently used phrases in messages classified with class 1 or 2
*   Transform messages into frequency matrices



**Dictionary**

In [ ]:
toxic_phrases = {'unskilled': 0, 'useless': 1, 'fucking': 2, 'motherfucking': 3, 'noob': 4, 'slut': 5, 'fuck': 6, 'fked': 7, 'fucked': 8, 'miserable': 9, 'sobad': 10, 'youaresobad': 11, 
                 'doesnothing': 12, 'somad': 13, 'scumbag': 14, 'fu': 15, 'youlost': 16, 'youdidthis': 17, 'goplay': 18, 'barbie': 19, 'screw': 20, 'usuck': 21, 'yousuck': 22,
                 'shit': 23, 'wakeup': 24, 'uninstall': 25, 'justuninstall': 26, 'retard': 27, 'tard': 28, 'yourmom': 29, 'urmom': 30, 'drugs': 31, 'justleave': 32, 'muted': 33, 
                 'shitted': 34, 'gay': 35, 'pussy': 36, 'surrend': 37, 'english': 38, 'stfu': 39, 'playyourself': 40, 'ignore': 41, 'twat': 42, 'wrong': 43, 'bronze': 44, 'nooob': 45, 
                 'suck': 46, 'fuuck': 47, 'worstteam': 48, 'udone': 49, 'uarebad': 50, 'youarebad': 51, 'ubelongin': 52, 'youbelongin': 53, 'shutup': 54, 'fail': 55, 'fuckyou': 56, 
                 'fucku': 57, 'crap': 58, 'fucker': 59, 'ihate': 60, 'stupid': 61, 'mother': 62, 'gohome': 63, 'youblind': 64, 'kys': 65, 'lowelo': 66, 'idiot': 67, 'trash': 68, 
                 'fag': 69, 'fking': 70, 'suckhard': 71, 'uneedhelp': 72, 'youneedhelp': 73, 'didnothing': 74, 'hateu': 75, 'hateyou': 76, 'shame': 77, 'bitch': 78, 'worst': 79, 
                 'biggestnoob': 80, 'pugface': 81, 'fuking': 82, 'udumb': 83, 'fkin': 84, 'die': 85, 'uninstallpls': 86, 'dick': 87, 'suck': 88, 'brainless': 89, 'neverplay': 90, 
                 'fuckoff': 91, 'nigger': 92, 'yourdad': 93, 'cock': 94, 'godie': 95, 'aids': 96, 'african': 97, 'fat': 98, 'dumbass': 99, 'pile': 100, 'poop': 101, 'nigga': 102, 
                 'poo': 103, 'ass': 104, 'nub': 105, 'whatanoob': 106, 'goingafk': 107, 'ufeeder': 108, 'stoptalkin': 109, 'braindead': 110, 'wrongwithu': 111, 'justbad': 112, 
                 'youtried': 113, 'pathetic': 114, 'awful': 115, 'fcking': 116, 'fck': 117, 'cantplay': 118, 'moron': 119, 'goafk': 120, 'shutit': 121, 'justmute': 122, 'whore': 123, 
                 'muteher': 124, 'mutehim': 125, 'feeder': 126, 'yourfault': 127, 'betterthanyou': 128, 'piss': 129, 'fuckingnoob': 130, 'yourmother': 131, 'delete': 132, 'suckhard': 133, 
                 'noskill': 134, 'girl': 135, 'lowlife': 136, 'dontlikeyou': 137, 'dontlikeu': 138, 'pieceofshit': 139, 'badplayer': 140, 'fucktard': 141, 'getlost': 142, 'getcarried': 143, 
                 'gtfo': 144, 'mental': 145, 'youdont': 146, 'sucksohard': 147, 'youcanmute': 148, 'shutthefuckup': 149, 'puss': 150, 'liar': 151, 'faggot': 152, 'shesbad': 153, 'sucking': 154, 
                 'lowlevel': 155, 'bequiet': 156, 'asshole': 157, 'cry': 158, 'prostitute': 159, 'german': 160, 'youfeeder': 161, 'asianbitch': 162, 'juststop': 163, 'fuman': 164, 'tetris': 165}
toxic_phrase_list = []
for word in toxic_phrases:
  toxic_phrase_list.append(word)

In [ ]:
def process_message(message):
  """
  Normalize a message by removing spaces and converting to lowercase.
  """
  no_spaces = message.replace(" ", "")
  return no_spaces.lower()

In [ ]:
def transform_text(messages, phrase_list):
    """
    This function should create a numpy array that contains the number of times each phrase
    of the dictionary appears in each message. 
    Each row in the resulting array should correspond to each message 
    and each column should correspond to a word of the vocabulary.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use process_message on each message.

    Args:
        messages: A list of strings where each string is a chat message.
        word_dictionary: A python dict mapping phrases to integers.

    Returns:
        A numpy array marking the words present in each message.
        Where the component (i,j) is the number of occurrences of the
        j-th vocabulary phrase in the i-th message.
    """
    arr = np.zeros((len(messages), len(phrase_list)))
    for msg_idx in range(len(messages)):
        processed = process_message(messages[msg_idx])
        for phrase in phrase_list:
          if phrase in processed:
            count_occurr = processed.count(phrase)
            arr[msg_idx, phrase_list.index(phrase)] += (count_occurr)
    return arr

**Matrix**

In [ ]:
train_matrix = transform_text(messages1, toxic_phrase_list)

# Multinomial Naive Bayes Classifier



*   Calculate model parameters
*   Add Laplace smoothing
*   Compute accuracies on test sets
*   Try on different training sets; compare parameters
*   Try different thresholds for predictions



In [ ]:
def fit_naive_bayes_model(matrix, labels):
    """
    Return the model parameters (likelihood of each word given class label,
    prior probabilities)

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: A vector with labels for the training data

    Returns: The trained model parameters
    """
    V = matrix.shape[1] # number of words in dictionary
    N = matrix.shape[0] # number of total messages
    neutral_messages = matrix[labels == 0]
    toxic_messages = matrix[labels == 1]
    derog_messages = matrix[labels == 2]

    # find likelihood parameters: probabilities of each word given the class label
    likelihoods_class0 = np.zeros((V,))
    likelihoods_class1 = np.zeros((V,))
    likelihoods_class2 = np.zeros((V,))
    priors = []

    n_neutral_words = np.sum(neutral_messages)
    n_toxic_words = np.sum(toxic_messages)
    n_derog_words = np.sum(derog_messages)
    for w in range(V):
        likelihoods_class0[w] = (sum(neutral_messages[:,w]) + 1)/ (n_neutral_words + V)
        likelihoods_class1[w] = (sum(toxic_messages[:,w]) + 1)/ (n_toxic_words + V)
        likelihoods_class2[w] = (sum(derog_messages[:,w]) + 1)/ (n_derog_words + V)

    priors.append(neutral_messages.shape[0] / N)
    priors.append(toxic_messages.shape[0] / N)
    priors.append(derog_messages.shape[0] / N)

    return [likelihoods_class0, likelihoods_class1, likelihoods_class2, priors]

In [ ]:
def predict_from_naive_bayes_model(messages, model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containing the predictions from the model (int array of 0 or 1 values)
    """
    likelihoods_class0 = model[0]
    likelihoods_class1 = model[1]
    likelihoods_class2 = model[2]
    priors = model[3]
    V = matrix.shape[1]
    N = matrix.shape[0]
    log_prior0 = math.log(priors[0])
    log_prior1 = math.log(priors[1])
    log_prior2 = math.log(priors[2])
    preds = np.zeros((N,))
    for i in range(N):
        sum_class0 = 0
        sum_class1 = 0
        sum_class2 = 0
        for t in range(V):
            sum_class0 += matrix[i, t] * math.log(likelihoods_class0[t])
            sum_class1 += matrix[i, t] * math.log(likelihoods_class1[t])
            sum_class2 += matrix[i, t] * math.log(likelihoods_class2[t])
        prob_class0 = log_prior0 + sum_class0
        prob_class1 = log_prior1 + sum_class1
        prob_class2 = log_prior2 + sum_class2
        probs = [prob_class0, prob_class1, prob_class2]
        class_pred = probs.index(max(probs))
        preds[i] = class_pred
    return preds

In [ ]:
# run on processed matrix
params = fit_naive_bayes_model(train_matrix, labels1)
# save predicted outputs to a file
test_matrix = transform_text(test_messages1, toxic_phrase_list)
preds = predict_from_naive_bayes_model(test_messages1, params, test_matrix)
np.savetxt('predictions.txt', preds)
naive_bayes_accuracy = np.mean(preds == test_labels1)
print(naive_bayes_accuracy)

In [ ]:
confusion_matrix(test_labels1, preds)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(test_labels1, preds, average=None))
print(confusion_matrix(test_labels1, preds, normalize='true'))
print("Accuracy:",metrics.accuracy_score(test_labels1, preds))

In [ ]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in part-c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: A list of the top five most indicative words in sorted order with the most indicative first
    """
    top_five = []
    V = len(dictionary)
    likelihoods_class0 = model[0]
    likelihoods_class1 = model[1]
    log_ratios = np.zeros((V,))
    key_list = list(dictionary.keys())

    for i in range(V):
        ratio = likelihoods_class0[i] / likelihoods_class1[i]
        log_ratios[i] = math.log(ratio)
    sorted_idxs = np.argsort(log_ratios)
    top_five_idxs = sorted_idxs[-5:]
    top_five_idxs_sorted = np.flip(top_five_idxs)

    for idx in top_five_idxs_sorted:
        top_five.append(key_list[idx])
    return top_five

In [ ]:
get_top_five_naive_bayes_words(params, toxic_phrases)